## 2.3 Reconocimiento de Imágenes en CIFAR10

En esta sección se trabajara con un dataset muy utilizado para experimentar con reconocimiento de imágenes: CIFAR10. Se trata de un conjunto de 60.000 imágenes RGB de $32x32$ pixeles que contiene 10 clases de objetos (6000 ejemplos por clase). La versión utilizada viene separada en 50.000 ejemplos de entrenamiento y 10.000 casos de prueba. El conjunto de pruebas fue obtenido seleccionando 1.000 imágenes aleatorias de cada clase. Los datos restantes han sido ordenados aleatoriamente y están organizados en 5 bloques de entrenamiento (batches). Las clases son mutuamente excluyentes y corresponden a las siguientes
categorías: gato, perro, rana, caballo, pájaro, ciervo, avión, automóvil, camión y barco.


### 2.3.a. Construcción de funcion para cargar datos de entrenamiento, prueba y validación.

In [ ]:
#Carga de diccionarios
def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog','frog', 'horse', 'ship', 'truck']

In [2]:
#1.a
from scipy.misc import imread
import cPickle as pickle
import numpy as np
import os
from sklearn.cross_validation import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

def load_CIFAR_one(filename):
    with open(filename, 'rb') as f:
        datadict = pickle.load(f)
        X = datadict['data']
        Y = datadict['labels']
        return X, np.array(Y, dtype=int)

def load_CIFAR10(PATH):
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(PATH, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_one(f)
        xs.append(X)
        ys.append(Y)
    Xtemp = np.concatenate(xs)
    Ytemp = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_one(os.path.join(PATH, 'test_batch'))
    Xtr, Xv, Ytr, Yv = train_test_split(Xtemp, Ytemp, test_size=0.2, random_state=0)
    del Xtemp, Ytemp
    return Xtr, Ytr, Xte, Yte, Xv, Yv

Xtr, Ytr, Xte, Yte, Xv, Yv = load_CIFAR10('.') #you need to add Xval

Con la función load_CIFAR10() se genera las matrices de datos de entrenamiento $X_{tr}$ y $Y_{tr}$ de tamaño $50.000x32x32$, matrices de datos de prueba $X_{t}$ y $Y_{t}$ de tamaño $10.000x32x32$ y matrices de datos de validación $X_{v}$ y $Y_{v}$ de tamaño $10.000x32x32$.


### 2.3.b. Construcción de funcion para escalar apropiadamente las imágenes.

In [3]:
#3.b
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical

def scaler_function(Xtr,Xt,Xv,scale=True):
    scaler = StandardScaler(with_std=scale).fit(Xtr)
    Xtr_scaled = scaler.transform(Xtr)
    Xt_scaled = scaler.transform(Xt)
    Xv_scaled = scaler.transform(Xv)
    return Xtr_scaled, Xt_scaled, Xv_scaled

Xtr, Xte, Xv = scaler_function(Xtr,Xte,Xv)

Ytr = to_categorical(Ytr)
Yte = to_categorical(Yte)
Yv = to_categorical(Yv)

/home/bampo/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
Using TensorFlow backend.


Se define la funcion scaler_function() la cual escalara los datos de cada pixel en formato RGB.

### 2.3.c. Red neuronal para clasificación del problema CIFAR10.

In [8]:
#3.c
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

def do_NN(Xtr, Ytr, Xte, Yte, Xv, Yv):
    model = Sequential()
    model.add(Dense(100, input_dim=Xtr.shape[1], init='uniform', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(10, init='uniform', activation='softmax'))
    model.compile(optimizer=SGD(lr=0.05), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(Xtr, Ytr, nb_epoch=50, batch_size=32, verbose=0, validation_data=(Xv,Yv))
    
    scores = model.evaluate(Xtr, Ytr)
    train_acc = scores[1]
    scores = model.evaluate(Xv, Yv)
    val_acc = scores[1]
    scores = model.evaluate(Xte, Yte)
    test_acc = scores[1]
    
    print "\nAccuracy de train: %f"%(train_acc)
    print "Accuracy de validacion: %f"%(val_acc)
    print "Accuracy de test: %f"%(test_acc)
    
do_NN(Xtr, Ytr, Xte, Yte, Xv, Yv) #Se entrena y evalua la red neuronal con 1 capa oculta de 100 neuronas.

 9952/10000 [============================>.] - ETA: 0s
Accuracy de train: 0.819920
Accuracy de validacion: 0.820320
Accuracy de test: 0.820000


Luego de entrenar una red neuronal con 1 capa oculta de 100 neuronas, se obtuvo buenos resultados, para los datos de prueba se obtuvo un rendimiento de $81.99\%$, en los datos de prueba un $82\%$ y en los datos de validación un $82.03\%$.

### 2.3.d. Red neuronal para clasificación del problema CIFAR10 utilizando representaciones de histograma de color y descriptores HOG.

In [9]:
# Se vuelven a cargar los datos para realizar el siguiente experimento.
Xtr, Ytr, Xte, Yte, Xv, Yv = load_CIFAR10('.')
Ytr = to_categorical(Ytr)
Yte = to_categorical(Yte)
Yv = to_categorical(Yv)

In [10]:
#1.d
from top_level_features import color_histogram_hsv
from top_level_features import hog_features
from top_level_features import extract_features

features_train = extract_features(Xtr,[color_histogram_hsv]) #extrae histogramas de color
features_test = extract_features(Xte,[color_histogram_hsv]) #extrae histogramas de color
features_val = extract_features(Xv,[color_histogram_hsv]) #extrae histogramas de color

print Xtr.shape
print features_train.shape

(40000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
(40000, 3072)
(40000, 10)


In [11]:
do_NN(features_train, Ytr, features_test, Yte, features_val, Yv)

 9888/10000 [============================>.] - ETA: 0s
Accuracy de train: 0.900000
Accuracy de validacion: 0.900000
Accuracy de test: 0.900000


Al utilizar los datos de histogramas de color para las matrices $X_{tr}$, $X_{t}$ y $X_{v}$, se obtiene que el rendimiento al entrenar una red neuronal con 1 capa oculta de 100 neuronas es $90\%$ para los 3 conjuntos de datos.

In [12]:
features_train = extract_features(Xtr,[hog_features]) #extrae hog features
features_test = extract_features(Xte,[hog_features]) #extrae hog features
features_val = extract_features(Xv,[hog_features]) #extrae hog features

print Xtr.shape
print features_train.shape

(40000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
(40000, 3072)
(40000, 144)


In [10]:
do_NN(features_train, Ytr, features_test, Yte, features_val, Yv)

 9824/10000 [============================>.] - ETA: 0s
Accuracy de train: 0.939097
Accuracy de validacion: 0.923510
Accuracy de test: 0.923050


Por otro lado, al utilizar los datos de HOG para las matrices $X_{tr}$, $X_{t}$ y $X_{v}$, se obtiene que el rendimiento al entrenar una red neuronal con 1 capa oculta de 100 neuronas es $93.9\%$ para los datos de entrenamiento, $92.3\%$ para los datos de prueba y $92.35\%$ para los datos de validación.

In [11]:
features_train = extract_features(Xtr,[hog_features, color_histogram_hsv]) #extrae todo
features_test = extract_features(Xte,[hog_features, color_histogram_hsv]) #extrae todo
features_val = extract_features(Xv,[hog_features, color_histogram_hsv]) #extrae todo

print Xtr.shape
print features_train.shape

(40000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
(40000, 3072)
(40000, 154)


In [12]:
do_NN(features_train, Ytr, features_test, Yte, features_val, Yv)

 9536/10000 [===========================>..] - ETA: 0s
Accuracy de train: 0.941872
Accuracy de validacion: 0.926120
Accuracy de test: 0.924690


Por último, al utilizar los datos de histogramas de color y los datos de HOG en conjunto para las matrices $X_{tr}$, $X_{t}$ y $X_{v}$, se obtiene que el rendimiento al entrenar una red neuronal con 1 capa oculta de 100 neuronas es $94.18\%$ para los datos de entrenamiento, $92.46\%$ para los datos de prueba y $92.61\%$ para los datos de validación.

Como se puede notar, en los 3 experimentos con distintos datos extraídos, el rendimiento fue bueno, donde el mínimo rendimiento fue $90\%$ y el modelo que obtuvo los mejores resultados para los datos de entrenamiento, de prueba y de validación fue utilizando todas las representaciones simultáneamente.


### 2.3.e. SVM no lineal para clasificación del problema CIFAR10.

In [14]:
# Se vuelven a cargar los datos para el experimento con SVM, se utilizara solo 10.000 datos de entrenamiento.
Xtr, Ytr, Xte, Yte, Xv, Yv = load_CIFAR10('.')
Xtr = Xtr[0:10000]
Ytr = Ytr[0:10000]
Xtr, Xte, Xv = scaler_function(Xtr,Xte,Xv)

In [15]:
from sklearn.svm import SVC

def do_SVM_no_lineal(x,y,xt,yt,xv,yv,model_type,best):
    Cs = np.logspace(-2, 4,base=2, num=7)
    acc_val = []
    models = []
    for C in Cs:
        if model_type == 'rbf':
            clf = SVC(C=C, kernel='rbf')
        elif model_type == 'poly':
            clf = SVC(C=C, kernel='poly',degree=2, coef0=1)

        clf=clf.fit(x,y)
                
        acc_val.append(clf.score(xv,yv))
        models.append(clf)

    if best:
        best_ = acc_val.index(max(acc_val))
        C_b = Cs[best_]
        model_b = models[best_]
        print "Mejor parámetro de regularización C: %s"%(C_b)        
        print "\nAccuracy de entrenamiento: %f"%(model_b.score(x,y))
        print "Accuracy de prueba: %f"%(model_b.score(xt,yt))
        print "Mejor Accuracy de validación: %f"%(max(acc_val))
        
        plt.figure(figsize=(10,5))
        ax = plt.gca()
        ax.plot(Cs,acc_val,label = 'Accuracy validación')
        plt.xlabel('C', fontsize = 16)
        plt.ylabel('Accuracy', fontsize = 16)
        plt.title('SVM no Lineal con kernel %s'%(model_type), fontsize = 16)
        ax.set_xscale('linear')
        plt.show()
        
    return 0

In [ ]:
do_SVM_no_lineal(Xtr, Ytr, Xte, Yte, Xv, Yv, 'rbf', best=True)

<img src="imagen2.png">

Dado los resultados de entrenar una SVM no lineal con kernel rbf, el mejor parametro C fue 4, con esto es obtuvo un accuracy de entrenamiento de $95.72\%$, un accuracy de prueba de $48.71\%$ y un accuracy de validación de $48.97\%$. Como puede notarse solo los datos de entrenamiento tuvieron buen rendimiento, esto es un claro caso de overfitting.

In [ ]:
do_SVM_no_lineal(Xtr, Ytr, Xte, Yte, Xv, Yv, 'poly', best=True)

In [ ]:
features_train = extract_features(Xtr,[color_histogram_hsv]) #extrae histogramas de color
features_test = extract_features(Xte,[color_histogram_hsv]) #extrae histogramas de color
features_val = extract_features(Xv,[color_histogram_hsv]) #extrae histogramas de color

In [ ]:
do_SVM_no_lineal(features_train, Ytr, features_test, features_val, Xv, Yv, 'rbf', best=True)

In [ ]:
do_SVM_no_lineal(features_train, Ytr, features_test, features_val, Xv, Yv, 'poly', best=True)

In [ ]:
features_train = extract_features(Xtr,[hog_features]) #extrae hog features
features_test = extract_features(Xte,[hog_features]) #extrae hog features
features_val = extract_features(Xv,[hog_features]) #extrae hog features

In [ ]:
do_SVM_no_lineal(features_train, Ytr, features_test, features_val, Xv, Yv, 'rbf', best=True)

In [ ]:
do_SVM_no_lineal(features_train, Ytr, features_test, features_val, Xv, Yv, 'poly', best=True)

In [ ]:
features_train = extract_features(Xtr,[hog_features, color_histogram_hsv]) #extrae todo
features_test = extract_features(Xte,[hog_features, color_histogram_hsv]) #extrae todo
features_val = extract_features(Xv,[hog_features, color_histogram_hsv]) #extrae todo

In [ ]:
do_SVM_no_lineal(features_train, Ytr, features_test, features_val, Xv, Yv, 'rbf', best=True)

In [ ]:
do_SVM_no_lineal(features_train, Ytr, features_test, features_val, Xv, Yv, 'poly', best=True)

Debido a que la ejecución de esta sección requiere mucho tiempo de computo, no se alcanzó a tener más resultados.

### 2.3.f. Árbol de clasificación de múltiples niveles para el problema CIFAR10.

In [16]:
Xtr, Ytr, Xte, Yte, Xv, Yv = load_CIFAR10('.')
from sklearn.tree import DecisionTreeClassifier as Tree

def do_Tree(x,y,xt,yt,xv,yv):
    N_ts = np.linspace(2, 20, num=20)
    acc_val = []
    models = []
    for n_t in N_ts:
        clf=Tree(criterion='gini',splitter='best',random_state=0,max_depth=n_t)
        clf=clf.fit(x,y)
        acc_val.append(clf.score(xv,yv))
        models.append(clf)
    best_ = acc_val.index(max(acc_val))            
    model_best = models[best_]
    
    print "Profundidad del mejor árbol = %d"%model_best.tree_.max_depth
    print "Mejor Accuracy de validación: %f"%(max(acc_val))
    print "\nAccuracy de entrenamiento: %f"%(model_best.score(x,y))
    print "Accuracy de prueba: %f"%(model_best.score(xt,yt))
    
    plt.figure(figsize=(10,5))
    ax = plt.gca()
    ax.plot(N_ts,acc_val,label = 'Accuracy validación')
    plt.xlabel('N_t', fontsize = 16)
    plt.ylabel('Accuracy', fontsize = 16)
    plt.title('Árbol de clasificacion con múltiples niveles', fontsize = 16)
    ax.set_xscale('linear')
    plt.show()
        
    return 0

In [ ]:
do_Tree(Xtr, Ytr, Xte, Yte, Xv, Yv)

<img src="imagen3.png">

Dado los resultados de entrenar un arbol de clasificación de multiples niveles desde 2 hasta 20 niveles. El mejor arbol fue el de profundidad 9, con esto es obtuvo un accuracy de entrenamiento de $39.0\%$, un accuracy de prueba de $30.2\%$ y un accuracy de validación de $29.83\%$. Como puede notarse el rendimiento de clasificar el problema de CIFAR10 mediante arboles no genera buenos resultados.

In [ ]:
features_train = extract_features(Xtr,[color_histogram_hsv]) #extrae histogramas de color
features_test = extract_features(Xte,[color_histogram_hsv]) #extrae histogramas de color
features_val = extract_features(Xv,[color_histogram_hsv]) #extrae histogramas de color

do_Tree(features_train, Ytr, features_test, Yte, features_val, Yv)

In [ ]:
features_train = extract_features(Xtr,[hog_features]) #extrae hog features
features_test = extract_features(Xte,[hog_features]) #extrae hog features
features_val = extract_features(Xv,[hog_features]) #extrae hog features

do_Tree(features_train, Ytr, features_test, Yte, features_val, Yv)

In [ ]:
features_train = extract_features(Xtr,[hog_features, color_histogram_hsv]) #extrae todo
features_test = extract_features(Xte,[hog_features, color_histogram_hsv]) #extrae todo
features_val = extract_features(Xv,[hog_features, color_histogram_hsv]) #extrae todo

do_Tree(features_train, Ytr, features_test, Yte, features_val, Yv)

Debido a que la ejecución de esta sección requiere mucho tiempo de computo, no se alcanzó a tener más resultados.